In [160]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [161]:
df = pd.read_csv('iris_data.csv', delimiter=',')

In [162]:
# Remove ID column
data = df.drop(columns=['Id'])

In [163]:
"""
Input Columns: SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm
Target Columns: SetosaScore, VersicolorScore, VirginicaScore
"""
x = data.drop(columns=['SetosaScore', 'VersicolorScore', 'VirginicaScore']).values
y = df.loc[0:, ['SetosaScore', 'VersicolorScore', 'VirginicaScore']].values

In [164]:
# pip install torch scikit-learn
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [165]:
"""

Additional guidelines are as follows:

Ensure the training loss and the test loss are less than 0.27. 
Use only Linear layers in the neural network.
Do not use any activation functions like ReLU or others
Do not use any regularization
Use Adam optimizer
Use MAE (Mean Absolute Loss)
Split the dataset to train-test with a ratio of 80% for training and 20% for testing.
"""

'\n\nAdditional guidelines are as follows:\n\nEnsure the training loss and the test loss are less than 0.27. \nUse only Linear layers in the neural network.\nDo not use any activation functions like ReLU or others\nDo not use any regularization\nUse Adam optimizer\nUse MAE (Mean Absolute Loss)\nSplit the dataset to train-test with a ratio of 80% for training and 20% for testing.\n'

In [166]:
# Split the data into training and test sets (80/20 split)]
x_train, x_test, y_train, y_test = \
train_test_split(x, y, test_size=0.2, random_state=42)
#  Specifies the proportion of the dataset that 
# should be allocated to the test set. 
# Here, 0.2 means 20% of the data will be used
# for testing, and the remaining 80% will be 
# used for training.

In [167]:
# Standarize the data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# StandardScaler: This is a class from 
# Scikit-learn used to standardize 
# features by removing the mean and 
# scaling to unit variance.
# Standardization: This process involves
# rescaling the features so that they have 
# a mean of 0 and a standard deviation of 1.
# This is important for many machine learning
# algorithms that perform better when 
# features are on a similar scale.

# This process is crucial for ensuring 
# that the model performs consistently,
# as many machine learning algorithms 
# assume or perform better when the 
# input features are on a similar scale.

In [168]:
# Convert the data to PyTorch sensors
x_train_tensor = torch.tensor(
    x_train, dtype=torch.float32
)
# torch.tensor(X_train): Converts 
# the X_train data (which is likely 
# a NumPy array or a Pandas 
# DataFrame) into a PyTorch tensor.
# dtype=torch.float32: Specifies that 
# the data type of the tensor should 
# be float32, which is a common 
# choice for numerical data in 
# deep learning models.
# Result: X_train_tensor is now a 
# PyTorch tensor containing the 
# standardized training input data, 
# ready for use in a neural network.

y_train_tensor = torch.tensor(
    y_train, dtype=torch.float32)

x_test_tensor = torch.tensor(
    x_test, dtype=torch.float32
)

y_test_tensor = torch.tensor(
    y_test, dtype=torch.float32)

In [169]:
# Create DataLoader for batching
train_dataset = TensorDataset(
    x_train_tensor, y_train_tensor
)
train_loader = DataLoader(
    train_dataset, batch_size=15, shuffle=True
)

In [170]:
# Define the neural network model
#  defines a new class NeuralNetwork 
# that inherits from nn.Module. 
# By inheriting from nn.Module, 
# the NeuralNetwork class gains access 
# to all the functionalities provided 
# by PyTorch for building and managing neural networks.
class NeuralNetwork(nn.Module):
    # Start by calling the constructor of 
    # the parent class (nn.Module), ensuring 
    # that the class is properly initialized.
    def __init__(self) -> None: # Constructor
        super(NeuralNetwork, self).__init__() 
        # nn.Linear creates a fully connected (linear) layer.
        self.fc1 = nn.Linear(x_train.shape[1], 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 3)

    # The forward method defines the forward pass of 
    # the network. It specifies how the input tensor
    # x should flow through the layers of the network.
    def forward(self, x):
        x = self.fc1(x) # No activation after the first layer
        x = self.fc2(x) # No activation afer the second layer
        x = self.fc3(x) # No activation afer the second layer
        x = self.fc4(x) # Output layer

        return x

In [ ]:
# Initialize the model, loss function, and optimizer
model = NeuralNetwork()
criterion = nn.L1Loss() # for mean absolute loss

optimizer = optim.Adam(
    model.parameters(), lr= 0.001 #learning rate of 0.01
)

In [176]:
# Training loop
epochs = 300
for epoch in range(epochs):
    # Set the model to training mode. This is 
    # important because certain layers, such 
    # as dropout or batch normalization, behave
    # differently during training than during 
    # evaluation. model.train() ensures that 
    # these layers are in training mode.
    model.train()
    total_loss = 0 # Initialize total loss for the epoch

    for batch_x, batch_y in train_loader:
        # Resets the gradients of all model parameters
        # to zero before starting the backpropagation
        # process for the current batch. This is 
        # important because gradients are accumulated
        # by default in PyTorch, so they need to be
        # cleared out before calculating the gradients
        # for the current batch
        optimizer.zero_grad()
        predictions = model(batch_x)
        # Compute the loss between the model's 
        # predictions and the actual target values 
        # (batch_y).
        loss = criterion(predictions ,batch_y)
        # Compute the gradients of the loss with 
        # respect to each model parameter using 
        # backpropagation. These gradients are 
        # used to update the model parameters 
        # in the next step.
        loss.backward()

        optimizer.step()

        total_loss += loss.item() # Accumulate the loss for each batch

    average_loss = total_loss / len(train_loader) # Compute the average loss for the epoch    
    print(f'Epoch: {epoch+1}, Average Loss: {average_loss:.4f}')



Epoch: 1, Average Loss: 0.2656
Epoch: 2, Average Loss: 0.2657
Epoch: 3, Average Loss: 0.2655
Epoch: 4, Average Loss: 0.2651
Epoch: 5, Average Loss: 0.2656
Epoch: 6, Average Loss: 0.2653
Epoch: 7, Average Loss: 0.2653
Epoch: 8, Average Loss: 0.2652
Epoch: 9, Average Loss: 0.2649
Epoch: 10, Average Loss: 0.2637
Epoch: 11, Average Loss: 0.2642
Epoch: 12, Average Loss: 0.2647
Epoch: 13, Average Loss: 0.2647
Epoch: 14, Average Loss: 0.2645
Epoch: 15, Average Loss: 0.2646
Epoch: 16, Average Loss: 0.2637
Epoch: 17, Average Loss: 0.2649
Epoch: 18, Average Loss: 0.2644
Epoch: 19, Average Loss: 0.2648
Epoch: 20, Average Loss: 0.2648
Epoch: 21, Average Loss: 0.2639
Epoch: 22, Average Loss: 0.2644
Epoch: 23, Average Loss: 0.2650
Epoch: 24, Average Loss: 0.2648
Epoch: 25, Average Loss: 0.2647
Epoch: 26, Average Loss: 0.2649
Epoch: 27, Average Loss: 0.2642
Epoch: 28, Average Loss: 0.2642
Epoch: 29, Average Loss: 0.2651
Epoch: 30, Average Loss: 0.2649
Epoch: 31, Average Loss: 0.2648
Epoch: 32, Averag

In [178]:
# Evaluate the model on the test set
# Switch to Evaluation Mode. In this mode, 
# certain layers like dropout and batch normalization, 
# which behave differently during training, will 
# operate in evaluation mode, meaning they won't 
# apply dropout or update running statistics.
# Why Use eval()?: This ensures that the 
# model's behavior is consistent during 
# testing and that the evaluation reflects
# the true performance on unseen data.
model.eval()

# Disabling Gradient Calculation.
# The torch.no_grad() context manager 
# temporarily disables gradient computation. 
# Since gradients are only necessary during 
# training (when you need to update the 
# model's parameters), disabling them 
# during evaluation saves memory and 
# computational resources because
# pytorch will not track the operations
# for that it might need later for gradient
# computation.
with torch.no_grad():
    test_predictions = model(x_test_tensor)
    test_loss = criterion(
        test_predictions, y_test_tensor 
    )
    print(f'Test loss: {test_loss.item():.4f}')

Test loss: 0.2030


In [179]:
[test_predictions, y_test_tensor]

[tensor([[ 0.9365, -0.1774,  0.1881],
         [ 0.0193,  0.0223,  0.9878],
         [-0.1601,  0.4076,  0.7332],
         [ 1.1161, -0.2728,  0.0964],
         [-0.1235,  0.4665,  0.6038],
         [ 0.1379,  0.2769,  0.5351],
         [-0.0412,  0.4899,  0.5216],
         [ 0.8188,  0.2248, -0.1306],
         [ 1.0081, -0.2252,  0.1807],
         [ 0.1610,  0.4317,  0.3657],
         [ 0.0424,  0.6347,  0.2277],
         [-0.4904,  0.8226,  0.6318],
         [ 1.0204, -0.1903,  0.1222],
         [-0.0812,  0.0865,  0.9926],
         [ 1.1653, -0.3515,  0.1499],
         [-0.0961,  0.7515,  0.2733],
         [-0.1032,  0.1685,  0.9452],
         [ 0.8758,  0.0150,  0.0552],
         [ 0.9022, -0.1385,  0.2090],
         [ 0.8085,  0.2206, -0.1183],
         [ 0.9721,  0.0728, -0.0988],
         [ 0.9004,  0.1059, -0.0782],
         [-0.0260, -0.0853,  1.1556],
         [ 0.7987,  0.3128, -0.1947],
         [ 0.8597,  0.1792, -0.1143],
         [ 0.8798,  0.0994, -0.0600],
         [ 0

In [182]:
def print_weights_biases(model):
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f'Layer: {name}')
            print(f'Values:\n{param.data}\n')

print_weights_biases(model=model)

Layer: fc1.weight
Values:
tensor([[ 0.0574,  0.0112,  0.0738, -0.1397],
        [-0.3066, -0.1162, -0.2040,  0.4437],
        [ 0.1776,  0.0868,  0.2685, -0.0862],
        [-0.3237, -0.1017,  0.2918,  0.0324],
        [ 0.0139, -0.0123, -0.0874,  0.0684],
        [-0.2646,  0.0767,  0.3397, -0.2831],
        [ 0.1552,  0.0249,  0.0565, -0.2315],
        [ 0.0647,  0.4042, -0.0141,  0.2987],
        [ 0.1031, -0.0268,  0.0006, -0.1075],
        [ 0.2562,  0.2837,  0.2302,  0.2602],
        [-0.0154, -0.0035, -0.0815,  0.1005],
        [-0.0462, -0.3333, -0.1279, -0.4275],
        [-0.1265,  0.1532,  0.0125, -0.0129],
        [-0.2139, -0.0824,  0.1721,  0.0264],
        [-0.3658,  0.2446,  0.0938,  0.2989],
        [-0.4175, -0.2673,  0.4326, -0.0624],
        [-0.0937, -0.0523,  0.4059, -0.3648],
        [ 0.1386, -0.0597, -0.2899,  0.2596],
        [ 0.2847,  0.1455, -0.2888,  0.1852],
        [ 0.0704, -0.0315, -0.1535,  0.0898],
        [ 0.2091, -0.1407, -0.3099,  0.0677],
        

In [183]:
my_test_data = [[6.7, 2.5, 6, 1.2],
            [5.4, 3.4, 1.5, 0.4],
            ]

# Transform data uring scaler
my_data_transformed = scaler.transform(my_test_data)

# Turn transformed data into tensor
my_data_tensor = torch.tensor(
    my_data_transformed, dtype=torch.float32
)

In [191]:
model.eval()
with torch.no_grad():
    predictions = model(my_data_tensor)

for i, prediction in enumerate(predictions):
    print(f'Prediction {i + 1}:')
    print(f'- Setosa Score: {prediction[0]:.4f}')
    print(f'- Versicolor Score: {prediction[1]:.4f}')
    print(f'- Virginia Score: {prediction[2]:.4f}')

Prediction 1:
- Setosa Score: -0.3148
- Versicolor Score: 1.1068
- Virginia Score: 0.0612
Prediction 2:
- Setosa Score: 0.8965
- Versicolor Score: 0.0592
- Virginia Score: 0.0009


In [185]:
my_test_data

[[6.7, 2.5, 6, 1.2], [5.4, 3.4, 1.5, 0.4]]

In [186]:
my_data_transformed

array([[ 1.03417754, -1.20786457,  1.2632157 , -0.08154786],
       [-0.59215004,  0.99195623, -1.44690142, -1.18416683]])